# Conversion of Data to a Machine Learning Friendly Format
This notebook demonstrates taking a single NetCDF file and converting the file into analysis ready numpy arrays stored in a zarr file for later use in neural network training.

Specifically, after having loaded multiple NetCDF files from UM model data into a single Iris CubeList, and saving this CubeList to disk, this notebook will:<ul>
<li>Load the single NetCDF file back from disk.</li>
<li>Extract the desired cubes: cloud volume fraction, specific humidity, air pressure, and air temperature.</li>
<li>Combine cubes of the same feature where metadata differences have prevented concatenation.</li>
<li>Convert the cubes to numpy arrays.</li>
<li>Format the arrays into a desirable dimension: (Sample Number, Height Level, Feature).</li>
<li>Generate data for the desired target we want to make a prediction on (cloud base height at a level in a sample).</li>
<li>Normalize data where necessary.</li>
<li>Save the data to disk for later loading to perform ML tasks.</li></ul> 


Define imports:

In [1]:
import os
import pathlib
import re
import iris
import dask

import numpy as np

Define file paths:

In [2]:
root_data_directory = pathlib.Path(os.environ['SCRATCH']) / 'cbh_data'

paths_to_load = root_data_directory / 'dev' / 'dev_large.nc' # one large nc file of iris' concatenation of all small nc files
path_to_save_result = root_data_directory / 'analysis_ready' / 'example_dev.npz' # ouput for numpy arrays
path_to_save_zarr = root_data_directory / 'analysis_ready' / 'example_dev.zarr' # output for zarr files

Settings for the notebook, with each constant given a comment above for a purpose description:

In [3]:
# generates a positional encoding array for a feature of each height layer in the sample
GENERATE_POSITIONAL_ENCODING_ARRAYS = False
# adds the height layer number to every feature vector in the input array
if GENERATE_POSITIONAL_ENCODING_ARRAYS:
    CONCATENATE_POSITIONAL_ENCONDING_TO_FEATURE_VECTOR = False

# realises the input array computation in two halves to avoid memory constraints of large computation
COMPUTE_INPUT_ARRAY_IN_HALVES = False

FREE_UP_MEMORY_AFTER_TARGET_COMPUTATION = False

# show all samples where clouds exist in the final layer (none)
# the final layer is used as the desired classification in the case of no cloud base existance prediction
VERIFY_NO_FINAL_LAYER_CLOUDS = False

# do extra compute to find the number of samples with cloud bases in the dataset
COMPUTE_CLOUD_BASE_SAMPLE_NUMBER = False

## Loading in the Cloud Base Height Data

In [4]:
cubes = iris.load(str(paths_to_load))
    
print('Find files complete, list of paths:', paths_to_load)

Find files complete, list of paths: /scratch/hsouth/cbh_data/dev/dev_large.nc


Show cube names:

In [5]:
print('Cube names:\n', [str(cube.name()) for cube in cubes])

print('\n' + "Example of cube metadata:", cubes[2].summary())

Cube names:
 ['cloud_volume_fraction_in_atmosphere_layer', 'm01s05i250', 'air_pressure', 'air_temperature', 'convective_rainfall_flux', 'convective_snowfall_flux', 'specific_humidity', 'stratiform_rainfall_flux', 'stratiform_snowfall_flux', 'upward_air_velocity']

Example of cube metadata: air_pressure / (Pa)                 (model_level_number: 70; latitude: 480; longitude: 640)
    Dimension coordinates:
        model_level_number                             x             -               -
        latitude                                       -             x               -
        longitude                                      -             -               x
    Auxiliary coordinates:
        level_height                                   x             -               -
        sigma                                          x             -               -
    Scalar coordinates:
        forecast_period             12.0 hours
        forecast_reference_time     2017-07-01 00:00:00
 

## Preprocess the data

### Extract the desired cubes: cloud volume fraction, specific humidity, air pressure, and air temperature
Cloud volume fraction will be used as our target for the problem, and the rest of the cubes are used as input.

In [6]:
def create_dataset(cubes):
    list_of_input_cubes = [ 'air_temperature',
                             'air_pressure',
                             'specific_humidity' ]
    target_cube_name = ['cloud_volume_fraction_in_atmosphere_layer']

    target_cube = iris.cube.CubeList([cube for cube in cubes if (cube.long_name in target_cube_name)])
    inp_cube = iris.cube.CubeList([cube for cube in cubes if (cube.standard_name in list_of_input_cubes)])
    
    
    return inp_cube, target_cube

Call the function defined above and verify success:

In [7]:
inp_cube, tar_cube = create_dataset(cubes)

print('input cube:\n',inp_cube, '\n')
print('target cubes:\n',tar_cube)

input cube:
 0: air_pressure / (Pa)                 (model_level_number: 70; latitude: 480; longitude: 640)
1: air_temperature / (K)               (model_level_number: 70; latitude: 480; longitude: 640)
2: specific_humidity / (kg kg-1)       (model_level_number: 70; latitude: 480; longitude: 640) 

target cubes:
 0: cloud_volume_fraction_in_atmosphere_layer / (1) (model_level_number: 70; latitude: 480; longitude: 640)


### Combine cubes of the same feature where metadata differences have prevented concatenation, while also extracting the numpy array of each cube
if duplicate cubes exist, concatenate them using numpy to avoid metadata matching issues:

In [8]:
def concatenate_same_cubes(cube_list):
    
    cube_name_dictionary = {}
    
    for cube in cube_list:
        # print('start cube load')
        cube_np_array = cube.core_data()
        # print('end load')
        
        if not cube.long_name is None:
            cube_name = cube.long_name
        elif not cube.standard_name is None:
            cube_name = cube.standard_name
        else:
            raise Exception('No name found on cube')
            
        try:
            # concat along the differing axis, forcast reference time
            cube_name_dictionary[cube_name] = np.concatenate((cube_np_array, cube_name_dictionary[cube_name]), axis=1)
            
            # print(cube_name_dictionary[cube_name].shape)
            
        except KeyError:
            cube_name_dictionary[cube_name] = cube_np_array
            
    return cube_name_dictionary

Call the function defined above and verify success:

In [9]:
inp_dict = concatenate_same_cubes(inp_cube)
tar_dict = concatenate_same_cubes(tar_cube)

print('Air Pressure array shape:', inp_dict['air_pressure'].shape)
print('Cloud Volume array shape:', tar_dict['cloud_volume_fraction_in_atmosphere_layer'].shape)
print('Array types:', type(inp_dict['air_pressure']))

Air Pressure array shape: (70, 480, 640)
Cloud Volume array shape: (70, 480, 640)
Array types: <class 'dask.array.core.Array'>


Combine dictionary elements to one array:

In [10]:
def combine_feats(dict_of_feats):
    
    add_dim_for_feature = [np.expand_dims(x, axis=0) for x in dict_of_feats.values()]
    feat_concat_array = np.concatenate(add_dim_for_feature, axis=0)
    return feat_concat_array

In [11]:
inp_array = combine_feats(inp_dict)
tar_array = combine_feats(tar_dict)

# verify and check dims
print('Dimensions to standardize for processing:')
print('Current Input Shape:', inp_array.shape)
print('Current Target Shape:', tar_array.shape)

Dimensions to standardize for processing:
Current Input Shape: (3, 70, 480, 640)
Current Target Shape: (1, 70, 480, 640)


Expand the dimensions of 'short' arrays to work in flattening (this applies in practice the smaller dev set of the data):

In [12]:
if(len(inp_array.shape) == 4):
    time_time2_dims_to_add = [1,2]
    inp_array = np.expand_dims(inp_array, time_time2_dims_to_add)
    tar_array = np.expand_dims(tar_array, time_time2_dims_to_add)
    print('New and correct shapes (should be 6 dims):')
    print(inp_array.shape)
    print(tar_array.shape)

New and correct shapes (should be 6 dims):
(3, 1, 1, 70, 480, 640)
(1, 1, 1, 70, 480, 640)


In [13]:
print('Show array storage metadata:')
inp_array

Show array storage metadata:


,Array,Chunk
Bytes,246.09 MiB,82.03 MiB
Shape,"(3, 1, 1, 70, 480, 640)","(1, 1, 1, 70, 480, 640)"
Count,15 Tasks,3 Chunks
Type,float32,numpy.ndarray


### Flatten the arrays

Flatten time and lat/long down to a single dimension, sample number </br>
Function expects 6-d array where each expected dimension is named in the function - cube_num, time, time2, height, lat, long

In [14]:
def flatten_cubes_with_numpy(np_array):
    
    # print('input dimensions:', np_array.shape)
    
    cube_num, time, time2, height, lat, long = np_array.shape
    
    # # verify shape
    # print(np_array.shape)
    
    # swap axis of time and height to ensure flattening preserves height
    cube_array = np_array.transpose(0,3,1,2,4,5)
    cubes_flattened = np.reshape(cube_array, (cube_num, height,(lat*long*time*time2)))
    
    # print('new dimensions', cubes_flattened.shape)
    
    cube_to_return = cubes_flattened.T
    # remove unnecessary dimensions
    cube_to_return = cube_to_return.squeeze()
    return cube_to_return

In [15]:
dask.config.set({'array.slicing.split_large_chunks': False}) # allow the potentially large chunk of data

inp_array = flatten_cubes_with_numpy(inp_array)
tar_array = flatten_cubes_with_numpy(tar_array)

# print('verify squeeze')
print('Shapes of flattened and transposed arrays:')
print('Input:', inp_array.shape)
print('Target:', tar_array.shape)

Shapes of flattened and transposed arrays:
Input: (307200, 70, 3)
Target: (307200, 70)


Rechunk large data to ensure large chunks are reduced for easier handling in dask:

In [16]:
tar_array = dask.array.rechunk(tar_array, chunks='auto')
print('Rechunked array storage metadata for target:')
tar_array

Rechunked array storage metadata for target:


dask.array<getitem, shape=(307200, 70), dtype=float32, chunksize=(307200, 70), chunktype=numpy.ndarray>

In [17]:
inp_array = dask.array.rechunk(inp_array, chunks='auto')
print('Rechunked array storage metadata for input:')
inp_array

Rechunked array storage metadata for input:


dask.array<transpose, shape=(307200, 70, 3), dtype=float32, chunksize=(307200, 70, 1), chunktype=numpy.ndarray>

## Preprocess the data toward ML algorithm input

### Generate data for the target of cloud base at certain height

preprocess the target
for the target, we define a cloud existing in a height layer:
if the cloud volume fraction is greater than 2 out of possible 8 oktas </br>
the cell below finds the first occurrences where the cloud volume is greater than the threshold marking a 1 in the array location, and stores 0 otherwise. </br>
Later, the final height layer will be marker for samples without a cloud base

In [18]:
cloud_threshold = 2./8.
cloud_over_threshold = dask.array.where(tar_array>cloud_threshold)

In [19]:
%%time
# realize the values for the where condition (dask array to numpy array)
print('Start base found sample compute')
sample_with_cloud = cloud_over_threshold[0].compute()
print('Start sample index compute')
index_on_sample = cloud_over_threshold[1].compute()

Start base found sample compute
Start sample index compute
CPU times: user 394 ms, sys: 474 ms, total: 868 ms
Wall time: 656 ms


Remove repeat indicies, e.g. where there are multiple layers above the cloud threshold, we only want the first occurence in a sample (the base):

In [20]:
%%time
_, first_duplicate_indicies = np.unique(sample_with_cloud, return_index=True)

if COMPUTE_CLOUD_BASE_SAMPLE_NUMBER:
    print('Start duplicate indicies compute')
    first_duplicate_indicies = first_duplicate_indicies.compute()
    print('Number of cloud bases found:',first_duplicate_indicies.shape)
    print('Out of samples:', tar_array.shape[0])

CPU times: user 28.3 ms, sys: 34.1 ms, total: 62.4 ms
Wall time: 60.7 ms


For clouds where no base was found, add a marker at the final height layer 
(where no cloud volume over threshold appears in the data).

In [21]:
%%time

# encode the cloud in onehot vector
one_hot_encoded_bases = np.zeros(tar_array.shape)
one_hot_encoded_bases[sample_with_cloud[first_duplicate_indicies],index_on_sample[first_duplicate_indicies]] = 1
# mark the end (final layer) if no cloud base detected
flip = lambda booleanVal: not booleanVal
vflip = np.vectorize(flip)
one_hot_encoded_bases[np.where(vflip(np.any(one_hot_encoded_bases, axis=1)))[0], -1] = 1

# Now reduce vectors as if each height layer is treated as a class where the model will predict, onehot -> class label e.g. 0,0,1,0, -> 2
class_label_encoded_bases = np.argmax(one_hot_encoded_bases, axis=1)

CPU times: user 102 ms, sys: 68.3 ms, total: 171 ms
Wall time: 173 ms


In [22]:
print('Target as class label:', class_label_encoded_bases.shape)
print('Output dim:', one_hot_encoded_bases.shape)

Target as class label: (307200,)
Output dim: (307200, 70)


In [23]:
# optionally, free up some memory
if FREE_UP_MEMORY_AFTER_TARGET_COMPUTATION:
    del sample_with_cloud
    del cloud_over_threshold
    del first_duplicate_indicies
    del index_on_sample
    del tar_dict
    del tar_cube
    del cubes

### Normalize input data

For the normalization of input data: we first transpose the input array so that the feature dimension is at the top level of the array, and numpy has an easier time accessing all values of the same feature. Then all values are normalized by being scaled in the range \[0,1\]

(must investigate mistake relating to ptp of local datasets instead of global values and make changes)

In [24]:
%%time

inp_array = inp_array.T
inp_array = (inp_array - np.min(inp_array, axis=(1,2)).reshape((3,1,1))) / (np.ptp(inp_array, axis=(1,2)).reshape((3,1,1)))
inp_array = inp_array.T

# a 2 half compute used to avoid memory constraints
if COMPUTE_INPUT_ARRAY_IN_HALVES:
    half = int(len(inp_array) / 2)
    inp_array_1 = inp_array[:half].compute()
    len_first_half = inp_array_1.shape[0]
    
else:
    inp_array = inp_array.compute()
    print('Finished compute of input array normalization')
    # convert to regular array, after verifying mask does not identify any values
    # (print below gives 0 masked values)
    num_of_masked = np.ma.count_masked(inp_array)
    print('Number of masked values after computation:', num_of_masked)
    assert num_of_masked == 0
    # unmask, giving all masked values NaN (but no masked values)
    inp_array = np.ma.filled(inp_array, np.nan)

    # # verify dimensions
    # print(inp_array.shape)
    # # and verify type
    # print('type of unmasked array:', type(inp_array))

Finished compute of input array normalization
Number of masked values after computation: 0
CPU times: user 1.14 s, sys: 1.2 s, total: 2.34 s
Wall time: 1.85 s


In [25]:
%%time
# second half of memory constraint compute, see above cell
if COMPUTE_INPUT_ARRAY_IN_HALVES:
    inp_array_2 = inp_array[half:].compute()
    print('Array type after compute:', type(inp_array_2))
    num_of_masked = np.ma.count_masked(inp_array_2)
    print('Count of masked (unfilled) values:', num_of_masked)
    assert num_of_masked == 0
    inp_array_2 = np.ma.filled(inp_array_2, np.nan)
    print('Array type after compute:', type(inp_array_2))
    len_second_half = inp_array_2.shape[0]

    # verify 
    print(len_second_half + len_first_half)
    print(inp_array.shape)
    
    # combine halves
    inp_array = inp_array_1 = np.concatenate((inp_array_1, inp_array_2), axis=0)
    del inp_array_1
    del inp_array_2
    

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


Compute and unmask target array (cloud volume):

In [26]:
print('Current type of target array:', type(tar_array))
print('Target shape:', tar_array.shape)
tar_array = tar_array.compute()
print('Finished compute of target array')

num_of_masked = np.ma.count_masked(tar_array)
print('Number of masked values after computation:', num_of_masked)
assert num_of_masked == 0

# unmask
tar_array = np.ma.filled(tar_array, np.nan)

Current type of target array: <class 'dask.array.core.Array'>
Target shape: (307200, 70)
Finished compute of target array
Number of masked values after computation: 0


In [27]:
if VERIFY_NO_FINAL_LAYER_CLOUDS:
    # verify the claim that no cloud bases appear in the final layer
    # can be strengthened to, no clouds exist in the final layer (next line returns 0)
    print('list of clouds at final height level:', np.where(tar_array[:,-1]>cloud_threshold)) 

#### View the produced arrays which are ready to be saved

In [28]:
# show 5 samples, first 5 height layers only, displaying all features in the layer (features not indexed)
# (automatic numpy array display reduction is quite large for this array)
inp_array[0:5, 0:5, :]

array([[[0.6558385 , 0.43776825, 0.00130421],
        [0.6535534 , 0.44206008, 0.00150347],
        [0.6503736 , 0.4434907 , 0.00167781],
        [0.6463207 , 0.45422032, 0.002065  ],
        [0.64145505, 0.47353363, 0.00277824]],

       [[0.6559205 , 0.43776825, 0.00130421],
        [0.6536282 , 0.44563663, 0.00151932],
        [0.65044475, 0.45064378, 0.00176159],
        [0.64639425, 0.4620887 , 0.00215331],
        [0.6415262 , 0.48211733, 0.00282579]],

       [[0.6559844 , 0.43776825, 0.00132006],
        [0.6536885 , 0.4434907 , 0.00148988],
        [0.6505026 , 0.45064378, 0.00176159],
        [0.6464521 , 0.4620887 , 0.0021601 ],
        [0.6415829 , 0.481402  , 0.00282353]],

       [[0.65603024, 0.43705294, 0.00132459],
        [0.6537331 , 0.44206008, 0.00148082],
        [0.6505448 , 0.44992846, 0.00175027],
        [0.64649314, 0.4620887 , 0.00216237],
        [0.6416239 , 0.4806867 , 0.00282353]],

       [[0.6560628 , 0.43776825, 0.00133138],
        [0.6537656 , 0.442

In [29]:
class_label_encoded_bases

array([16, 15, 15, ...,  0,  0,  0])

In [30]:
tar_array

array([[0.140625, 0.125   , 0.09375 , ..., 0.      , 0.      , 0.      ],
       [0.140625, 0.109375, 0.09375 , ..., 0.      , 0.      , 0.      ],
       [0.140625, 0.125   , 0.09375 , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.90625 , 1.      , 1.      , ..., 0.      , 0.      , 0.      ],
       [1.      , 1.      , 1.      , ..., 0.      , 0.      , 0.      ],
       [0.953125, 1.      , 1.      , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)

In [31]:
one_hot_encoded_bases

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

#### Save a selection of wanted arrays (inp_array, tar_array, one_hot_encoded_bases)

Now to save the computed array </br>
(will not save one of class label output or one_hot as easy conversion between the two)</br>
(went with saving one-hot to emulate the data produced/used by base solution)

In [32]:
# verify input and output shapes
print('Input dim:', inp_array.shape)
print('Cloud Output dim:', tar_array.shape)

Input dim: (307200, 70, 3)
Cloud Output dim: (307200, 70)


In [33]:
%%time

print('Saving numpy arrays')

with open(path_to_save_result, 'w+b') as f:
    # variable assignment that name the arrays for the saved file
    input_x=inp_array
    output_onehot=one_hot_encoded_bases
    output_cloud_volume=tar_array

    np.savez(f, input_x=input_x, output_cloud_volume=output_cloud_volume, output_onehot=output_onehot)

Saving numpy arrays
CPU times: user 694 ms, sys: 155 ms, total: 848 ms
Wall time: 1.02 s


The following cell is code to create a positional encoding for the height layers in the data, e.g. the data at height layer 0 would have the positional encoding of: 0 as part of the input feature. It is commented out as PyTorch Dataloaders are found to have the capability to produce this information at load-time, which seems like a better option than creating a potentially huge array for each position that is scaled up to the size of the sample number redundantly.

In [34]:
# create an extra positional encoding optionally for input use
if GENERATE_POSITIONAL_ENCODING_ARRAYS:
    sample_num, height_dim, _ = inp_array.shape
    # generate height values
    height_position_vector = np.arange(height_dim)
    # extend dimensions out to match input feats
    height_position_vector = np.repeat([height_position_vector], sample_num, axis=0)

    # verify
    print('shape of encoding vector:', height_position_vector.shape)

    x,y = height_position_vector.shape
    # add a dimension for height to act as a feature
    height_position_vector = height_position_vector.reshape(x,y,1)

    # fit the dtype of the feature to match the dtype of other feats
    height_position_vector = height_position_vector.astype(inp_array.dtype)

    # combine height feature into input array 
    if CONCATENATE_POSITIONAL_ENCONDING_TO_FEATURE_VECTOR:
        inp_array = np.concatenate((height_position_vector, inp_array), axis=2, dtype=np.float32) #leave the concat for within the model after producing embedding

    # verify datatypes
    print('input dtype', inp_array.dtype)
    print('height encoding dtype', height_position_vector.dtype)

## Convert saved numpy arrays to zarr files

First, open the numpy files and load each array into a variable:

In [35]:
import zarr # (package not in cop torch conda env)

with open(path_to_save_result, 'r+b') as f:
    numpy_files = np.load(f)
    inp_arr = numpy_files['input_x']
    onehot_arr = numpy_files['output_onehot']
    tar_arr = numpy_files['output_cloud_volume']

Then, store the loaded numpy arrays into zarr, which will chunk and compress each array:

In [36]:
store = zarr.DirectoryStore(path_to_save_zarr)
# define objected for arrays to be grouped under

zarr_grouping = zarr.group(store=store, overwrite=True)

# initialize and then write on zarr arrays for all desired arrays to be saved

humidity_temp_pressure_x =  zarr_grouping.zeros(shape=inp_arr.shape, 
                                                dtype=inp_arr.dtype, 
                                                name='humidity_temp_pressure_x.zarr')
humidity_temp_pressure_x[:] = inp_arr

onehot_cloud_base_height_y = zarr_grouping.zeros(shape=onehot_arr.shape, 
                                                 dtype=onehot_arr.dtype, 
                                                 name='onehot_cloud_base_height_y.zarr')
onehot_cloud_base_height_y[:] = onehot_arr

cloud_volume_fraction_y = zarr_grouping.zeros(shape=tar_arr.shape, 
                                              dtype=tar_arr.dtype, 
                                              name='cloud_volume_fraction_y.zarr')
cloud_volume_fraction_y[:] = tar_arr

# output some summary for zarr
# view group values
printF = lambda obj: print(obj)
print("Elements of zarr group:")
zarr_grouping.visitvalues(printF)
# view group tree
print("\nTree of zarr group:\n", zarr_grouping.tree())
# see chunk size
print("\nShape array example:", humidity_temp_pressure_x.shape)
print("\nZarr chunking shape of the array:", humidity_temp_pressure_x.chunks)

Elements of zarr group:
<zarr.core.Array '/cloud_volume_fraction_y.zarr' (307200, 70) float32>
<zarr.core.Array '/humidity_temp_pressure_x.zarr' (307200, 70, 3) float32>
<zarr.core.Array '/onehot_cloud_base_height_y.zarr' (307200, 70) float64>

Tree of zarr group:
 /
 ├── cloud_volume_fraction_y.zarr (307200, 70) float32
 ├── humidity_temp_pressure_x.zarr (307200, 70, 3) float32
 └── onehot_cloud_base_height_y.zarr (307200, 70) float64

Shape array example: (307200, 70, 3)

Zarr chunking shape of the array: (38400, 9, 1)
